# Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №2 - Дерево решений


**Общая информация**

**Срок сдачи:** до 1 ноября 2018, 06:00 
**Штраф за опоздание:** -2 балла после 06:00 1 ноября, -4 балла после 06:00 8 ноября, -6 баллов после 06:00 15 ноября

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush
Необходимо в slack создать таск в приватный чат:   
/todo Фамилия Имя *ссылка на гитхаб* @alkhamush   
Пример:   
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw2.ipynb @alkhamush   

Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (4 балла)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. В комментариях, где написано "Что делает этот блок кода?", ответьте на этот вопрос. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на wine и Speed Dating Data.

###### Задание 2 (2 балла)
Добиться скорости работы fit такой, чтобы она была медленнее sklearn не более чем в 10 раз. Скорость проверяем на  wine и Speed Dating Data. Для ускорения используем только numpy.

###### Задание 3 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 3 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 4 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw2.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import six

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import BaseEstimator
from abc import ABCMeta

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

%matplotlib inline
# %load_ext pycodestyle_magic

d:\soft\devkits\anaconda_3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
def calc_gini(probability):
    if probability.ndim == 2:
        return 1 - (probability ** 2).sum(axis=1).reshape(-1, 1)
    else:
        return 1 - (probability ** 2).sum()


def calc_entropy(probability):
    if probability.ndim == 2:
        return - np.nan_to_num(probability * np.log(probability)).sum(axis=1).reshape(-1, 1)
    else:
        return - np.nan_to_num(probability * np.log(probability)).sum()


def calc_miscass(probability):
    if probability.ndim == 2:
        max_probability = probability.max(axis=1)
        return 1 - max_probability.reshape(-1, 1)
    else:
        max_probability = probability.max()
        return 1 - max_probability


class MyDecisionTreeClassifier(six.with_metaclass(ABCMeta, BaseEstimator)):
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None,
                 sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.class_amount = -1
        self.feature_importances_ = None
        self.criterion = criterion
        if criterion == 'gini':
            self.G_function = self.__gini
            self.G_sub_function = calc_gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
            self.G_sub_function = calc_entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
            self.G_sub_function = calc_miscass
        else:
            raise ValueError('invalid criterion name')

        self.max_features = max_features
        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            raise ValueError('invalid max_features name')

    # I = 1 - ΣPk^2
    def __gini(self, l_c, l_s, r_c, r_s):
        left_part = (l_s / (l_s + r_s))
        return (left_part * calc_gini(l_c / l_s) +
                (1 - left_part) * calc_gini(r_c / r_s))

    # - ΣPk*log(Pk)
    def __entropy(self, l_c, l_s, r_c, r_s):
        left_part = (l_s / (l_s + r_s))
        return (left_part * calc_entropy(l_c / l_s) +
                (1 - left_part) * calc_entropy(r_c / r_s))

    # 1 - max(Pk)
    def __misclass(self, l_c, l_s, r_c, r_s):
        left_part = (l_s / (l_s + r_s))
        return (left_part * calc_miscass(l_c / l_s) +
                (1 - left_part) * calc_miscass(r_c / r_s))

    # sqrt(n) случайных
    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:np.sqrt(n_feature)]

    # log2(n) случайных
    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:np.log2(n_feature)]

    # все n
    def __get_feature_ids_N(self, n_feature):
        return np.arange(n_feature)

    # сортируем данные по, убыванию
    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    # нужно найти такую величину, которая готова разбить наше множество наилучшим образом
    def __find_threshold(self, x, y):
        # сортируем по величине значения фичи
        sorted_x, sorted_y = self.__sort_samples(x, y)
        # количество классов
        class_amount = self.class_amount
        # число элементов, которые нужно отделить
        cut_size = np.int(self.min_samples_split / 2 - 1)

        # получаем внутренние элементы
        splitted_sorted_y = sorted_y[cut_size:-cut_size + sorted_y.size]
        # тут мы делим наши значения на группы
        # в r_ids попадают те номера, которые отличаются от последующего (словно "неправильные")
        r_border_ids = np.where(splitted_sorted_y[:-1] !=
                                splitted_sorted_y[1:])[0] + (cut_size + 1)
        # если дальше делить нет смысла
        # 1) элементов < чем минимум для деления
        # 2) или больше в группе нет "неправильных"
        if len(r_border_ids) == 0:
            return np.inf, None

        # считаем колличество подряд идущих элементов исходя из того, что r_border_ids - это позиции !=`х элементов
        # получаем, что разница 2х подряд идущих индексов есть количество подряд идущих равных элементов
        eq_el_count = r_border_ids - np.append(np.array([cut_size]),
                                               r_border_ids[:-1])
        # Инициализируем индикаторную матрицу "неправильных" элементов
        one_hot_code = np.zeros((r_border_ids.shape[0], class_amount))
        # На позициях тех классов, чьи элементы повторялись устанавливаем 1
        # для каждой строки (все строки одинаковы)
        one_hot_code[np.arange(r_border_ids.shape[0]),
                     sorted_y[r_border_ids - 1]] = 1
        # умножаем предыдущую матрицу на столбец подряд идущих элементов
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        # для 0 позиции добавляем число "вырезанных" элементов
        class_increments[0] = class_increments[0] + np.bincount(sorted_y[:cut_size], minlength=class_amount)

        # в левую вершину записываем "Хорошие" подряд идущие элементы (с помощью коммулятивной суммы)
        l_class_count = np.cumsum(class_increments, axis=0)
        # в правую остальные
        r_class_count = np.bincount(sorted_y, minlength=class_amount) - l_class_count
        # число элементов слева это номер на котором происходит последняя смена класса
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        # число остальных есть разница всего числа и числа для левой вершины
        r_sizes = sorted_y.shape[0] - l_sizes

        # считаем для каждого варианта его меру неопределенности
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        # выбираем лучшее
        idx = np.argmin(gs)

        # Забираем искомую величину - на каком элементе разобьем
        left_el_id = l_sizes[idx][0]
        # возвращаем неопределенность + значение, на котором нужно разбить выборку
        return gs[idx], (sorted_x[left_el_id - 1] + sorted_x[left_el_id]) / 2.0

    def __fit_node(self, x, y, node_id, depth, pred_f=-1):
        # clear prev
        if node_id == 0:
            self.tree.clear()
        # stop if empty
        if y.size == 0:
            return

        # stop if:
        #   ready
        #   less then min
        #   depth
        #   % of max class in cur node is more then sufficient_share
        if y.size < self.min_samples_split or \
                depth == self.max_depth or \
                np.unique(y).shape[0] == 1 or \
                np.max(np.bincount(y)) / y.size >= self.sufficient_share:
            self.__init_leaf(x, y, node_id)
            return

        chosen_features = self.get_feature_ids(x.shape[1])

        # хранит показатель меры неопределенности + место разделения
        thresholds = np.zeros((2, x.shape[1]), dtype=np.float32)

        for i in range(x.shape[1]):
            if i in chosen_features:
                thresholds[:, i] = np.array(self.__find_threshold(x[:, i], y))

        # searching for best feature
        min_uncertainty = np.min(thresholds[0])
        best_features = np.where(thresholds[0] == min_uncertainty)[0]
        result = np.random.choice(best_features)
        threshold = thresholds[1, result]

        # divide samples
        x_l, x_r, y_l, y_r = self.__div_samples(x, y, result, threshold)

        if y_l.size == 0 or y_r.size == 0:
            self.__init_leaf(x, y, node_id)
            return

        self.__set_importance(y_l, y_r, result)
        self.__init_none_leaf(node_id, result, threshold)
        self.__fit_node(x_l, y_l, 2 * node_id + 1, depth + 1, result)
        self.__fit_node(x_r, y_r, 2 * node_id + 2, depth + 1, result)
        return

    def __init_leaf(self, x, y, node_id):
        node = [self.LEAF_TYPE]
        classes_count = np.bincount(y)
        node.append(np.argmax(classes_count))
        node.append(classes_count / classes_count.sum())
        self.tree.update({node_id: node})

    def __init_none_leaf(self, node_id, feature_id, threshold):
        node = [self.NON_LEAF_TYPE, feature_id, threshold]
        self.tree.update({node_id: node})

    def __set_importance(self, y_l, y_r, feature_id):
        classes_amount = np.append(y_l, y_r).max() + 1
        l_c = np.bincount(y_l, minlength=classes_amount)
        r_c = np.bincount(y_r, minlength=classes_amount)
        self.feature_importances_[feature_id] += \
            (self.G_sub_function((l_c + r_c) / (y_l.size + y_r.size)) -
             self.G_function(l_c, y_l.size, r_c, y_r.size))

    def fit(self, x, y):
        self.feature_importances_ = np.zeros((x.shape[1],), dtype=np.float32)
        self.class_amount = np.max(y) + 1
        self.__fit_node(x, y, 0, 0)

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]

    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])

    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

    #     def get_params(self, deep=True):
    #         out = dict()
    #         out['max_depth'] = self.max_depth
    #         out['sufficient_share'] = self.sufficient_share
    #         out['min_samples_split'] = self.min_samples_split
    #         out['criterion'] = self.criterion
    #         out['max_features'] = self.max_features
    #         print(out)
    #         return out

    #     def set_params(self, **params):
    #         print(**params)
    #         for key, value in params.items():
    #             self.__setattr__(key, value)
    #         return self

    def score(self, x, y):
        pred_y = self.predict(x)
        return 1.0 - (np.bincount(np.abs(y - pred_y))[1:]).sum() / y.size


In [3]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [4]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [5]:
%time clf.fit(X_train, y_train)

Wall time: 998 µs


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [6]:
%time my_clf.fit(X_train, y_train)

Wall time: 29.9 ms


## Проверка качества работы на wine

In [7]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.8407407407407407

In [8]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.890993265993266

## Подготовка данных Speed Dating Data 

In [9]:
df = pd.read_csv('Speed Dating Data.csv', encoding='cp1251')

In [10]:
df = df.iloc[:, :97]

In [11]:
df = df.drop(['id'], axis=1)
df = df.drop(['idg'], axis=1)
df = df.drop(['condtn'], axis=1)
df = df.drop(['round'], axis=1)
df = df.drop(['position', 'positin1'], axis=1)
df = df.drop(['order'], axis=1)
df = df.drop(['partner'], axis=1)
df = df.drop(['age_o', 'race_o', 'pf_o_att',
              'pf_o_sin', 'pf_o_int',
              'pf_o_fun', 'pf_o_amb', 'pf_o_sha',
              'dec_o', 'attr_o', 'sinc_o', 'intel_o', 'fun_o',
              'amb_o', 'shar_o', 'like_o', 'prob_o', 'met_o'],
             axis=1)
df = df.drop(['field'], axis=1)
df = df.drop(['undergra'], axis=1)
df = df.drop(['from', 'zipcode'], axis=1)
df = df.drop(['career'], axis=1)
df = df.drop(['sports', 'tvsports', 'exercise', 'dining', 'museums', 'art', 'hiking', 'gaming',
              'clubbing', 'reading', 'tv', 'theater', 'movies', 'concerts', 'music', 'shopping', 'yoga'], axis=1)
df = df.drop(['expnum'], axis=1)
df = df.drop(['wave'], axis=1)

df = df.dropna(subset=['age'])
df = df.dropna(subset=['imprelig', 'imprace'])
df = df.dropna(subset=['date'])

df.loc[:, 'mn_sat'] = df.loc[:, 'mn_sat'].str.replace(',', '').astype(np.float)
df.loc[:, 'tuition'] = df.loc[:, 'tuition'].str.replace(',', '').astype(np.float)
df.loc[:, 'income'] = df.loc[:, 'income'].str.replace(',', '').astype(np.float)

# df.int_corr = df.int_corr.fillna(method='ffill')
# df.field_cd = df.field_cd.fillna(method='ffill')
# df.amb1_1 = df.amb1_1.fillna(method='ffill')
# df.amb2_1 = df.amb2_1.fillna(method='ffill')
# df.amb3_1 = df.amb3_1.fillna(method='ffill')
# df.shar1_1 = df.shar1_1.fillna(method='ffill')
# df.shar2_1 = df.shar2_1.fillna(method='ffill')
# df.fun1_1 = df.fun1_1.fillna(method='ffill')
# df.fun3_1 = df.fun3_1.fillna(method='ffill')
# df.attr3_1 = df.attr3_1.fillna(method='ffill')
# df.sinc3_1 = df.sinc3_1.fillna(method='ffill')
# df.intel3_1 = df.intel3_1.fillna(method='ffill')

df.loc[:, 'mn_sat'] = df.mn_sat.fillna(df.mn_sat[df.mn_sat.notna()].mean())
df.loc[:, 'income'] = df.income.fillna(df.income[df.income.notna()].mean())
df.loc[:, 'tuition'] = df.tuition.fillna(df.tuition[df.tuition.notna()].mean())
df.loc[:, 'career_c'] = df.loc[:, 'career_c'].fillna(0)
df.loc[:, 'field_cd'] = df.loc[:, 'field_cd'].fillna(0)

for i in [4, 5]:
    feat = ['attr{}_1'.format(i), 'sinc{}_1'.format(i),
            'intel{}_1'.format(i), 'fun{}_1'.format(i),
            'amb{}_1'.format(i), 'shar{}_1'.format(i)]
    if i != 4:
        feat.remove('shar{}_1'.format(i))
    df = df.drop(feat, axis=1)

df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']].sum(axis=1)
df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']] = \
    (df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']].T / df.loc[:,
                                                                                      'temp_totalsum'].T).T * 100

df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']].sum(axis=1)
df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']] = \
    (df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']].T / df.loc[:,
                                                                                      'temp_totalsum'].T).T * 100

df = df.drop(['temp_totalsum'], axis=1)


In [12]:
df_male = df.query('gender == 1').drop_duplicates(subset=['iid', 'pid'])\
                                 .drop(['gender'], axis=1)\
                                 .dropna()
df_female = df.query('gender == 0').drop_duplicates(subset=['iid'])\
                                   .drop(['gender', 'match', 'int_corr', 'samerace'], axis=1)\
                                   .dropna()
        
df_female.columns = df_female.columns + '_f'

In [13]:
df_female = df_female.drop(['pid_f'], axis=1)

In [14]:
df_pair = df_male.join(df_female.set_index('iid_f'), 
                       on='pid', 
                       how='inner')
df_pair = df_pair.drop(['iid', 'pid'], axis=1)

In [15]:
df_pair.head()

,match,int_corr,samerace,age,field_cd,mn_sat,tuition,race,imprace,imprelig,...,sinc2_1_f,intel2_1_f,fun2_1_f,amb2_1_f,shar2_1_f,attr3_1_f,sinc3_1_f,fun3_1_f,intel3_1_f,amb3_1_f
100,0,0.14,0,27.0,8.0,1299.211122,21111.97234,2.0,7.0,3.0,...,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0
110,0,0.54,0,22.0,1.0,1299.211122,21111.97234,2.0,1.0,1.0,...,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0
120,1,0.16,1,22.0,1.0,1299.211122,21111.97234,4.0,3.0,5.0,...,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0
130,1,0.61,0,23.0,1.0,1299.211122,21111.97234,2.0,1.0,1.0,...,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0
140,1,0.21,0,24.0,1.0,1299.211122,21111.97234,3.0,3.0,1.0,...,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0


In [16]:
X = df_pair.iloc[:, 1:].values
y = df_pair.iloc[:, 0].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=123)

In [17]:
X_train.shape

(3599, 62)

In [18]:
X_test.shape

(400, 62)

## Проверка скорости работы на Speed Dating Data 

In [19]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [20]:
%time clf.fit(X_train, y_train)

Wall time: 124 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [21]:
%time my_clf.fit(X_train, y_train)

Wall time: 1.01 s


## Проверка качества работы на Speed Dating Data

In [22]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.5638909727431858

In [23]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.5

## Задание 3

In [24]:
list(clf.feature_importances_.argsort()[:10])

[40, 25, 30, 1, 13, 38, 6, 60, 31, 26]

In [25]:
list(my_clf.feature_importances_.argsort()[:10])

[30, 34, 26, 25, 24, 23, 22, 36, 42, 19]

## Задание 4

In [26]:
from scipy.stats import uniform
from scipy.stats import randint
from scipy.stats import uniform

try:
    from sklearn.model_selection import GridSearchCV
    from sklearn.model_selection import RandomizedSearchCV
    from sklearn.model_selection import StratifiedKFold
except ImportError:
    from sklearn.cross_validation import GridSearchCV
    from sklearn.cross_validation import RandomizedSearchCV
    from sklearn.cross_validation import StratifiedKFold

RND_SEED = 123

In [27]:
param_grid_clf = {
    'criterion': ['gini', 'entropy'],
    'max_depth': randint(2,10),
    'min_samples_split': randint(5, 10),
    'max_features': ['sqrt', 'log2', None]
}

In [28]:
param_grid_my_clf = {
    'criterion': ['gini', 'entropy', 'misclass'],
    'max_depth': randint(2,10),
    'min_samples_split': randint(5, 10),
    'sufficient_share': uniform(0, 1),
    'max_features': ['sqrt', 'log2', None]
}

In [29]:
cv = StratifiedKFold(n_splits=5, shuffle=True)

model = DecisionTreeClassifier()
random_search = RandomizedSearchCV(model, param_distributions=param_grid_clf, n_iter=200, n_jobs=-1,
                                   cv=cv, scoring='roc_auc')
random_search.fit(X, y)
print("not me")
print(random_search.best_params_)
print(random_search.best_score_)

not me
{'criterion': 'entropy', 'max_depth': 7, 'max_features': 'sqrt', 'min_samples_split': 6}
0.6231987492171501


In [ ]:
cv = StratifiedKFold(n_splits=5, shuffle=True)

model = MyDecisionTreeClassifier()
random_search = RandomizedSearchCV(model, param_distributions=param_grid_my_clf, n_iter=10, n_jobs=-1,
                                   cv=cv, scoring='roc_auc')
random_search.fit(X, y)
print("me")
print(random_search.best_params_)
print(random_search.best_score_)